In [2]:
pip install fastcoref

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fastcoref import TrainingArgs, CorefTrainer, LingMessCoref, FCoref
import os
import json
import random
import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
#%env

{'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs:/opt/conda/lib',
 'NV_LIBCUBLAS_VERSION': '11.11.3.6-1',
 'NV_NVPROF_DEV_PACKAGE': 'cuda-nvprof-11-8=11.8.87-1',
 'NV_CUDA_COMPAT_PACKAGE': 'cuda-compat-11-8',
 'PIP_ROOT_USER_ACTION': 'ignore',
 'MKL_THREADING_LAYER': 'GNU',
 'NV_CUDA_NSIGHT_COMPUTE_VERSION': '11.8.0-1',
 'KAGGLE_URL_BASE': 'https://www.kaggle.com',
 'KAGGLE_KERNEL_RUN_TYPE': 'Interactive',
 'MPLBACKEND': 'agg',
 'HOSTNAME': '8554ac929f35',
 'GCSFUSE_METADATA_IMAGE_TYPE': 'DLC',
 'SHLVL': '0',
 'LD_LIBRARY_PATH': '/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib',
 'GIT_COMMIT': '9206ba01f1497b13b7196a778c28657bacce6f79',
 'NV_LIBNCCL_PACKAGE_VERSION': '2.15.5-1',
 'ENABLE_MULTI_ENV': 'false',
 'HOME': '/root',
 'NV_CUDNN_PACKAGE_NAME': 'libcudnn8',
 'TENSORFLOW_VERSION': '2.12.0',
 'NV_LIBCUBLAS_DEV_VERSION': '11.11.3.6-1',
 'KMP_SETTINGS': '

In [4]:
#%env WANDB_MODE=offline

env: WANDB_MODE=offline


# Create Baseline predictions with FastCoref and LingMess

In [6]:
with open("/kaggle/input/tdac-coref-corpus/TDAC_dev_test_combined.jsonlines", 'r') as f:
    gold = [json.loads(l) for l in list(f)]
    
texts = []
for doc in gold:
    texts.append(doc['text'])

In [9]:
seeds = [k for k in range(0, 10)]

for seed in seeds:
    random.seed(seed)
    model = FCoref(model_name_or_path='biu-nlp/f-coref')
    start  = time.time()
    preds = model.predict(texts=texts, output_file=f'TDAC_FCoref_preds_on_test_corpus_seed_{seed}.jsonlines')
    print(f'prediction time : {time.time() - start} seconds')

print(f'mean time : {(time.time() - start)/len(seeds)} seconds')

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Inference:   0%|          | 0/15 [00:00<?, ?it/s]

prediction time : 3.1301114559173584 seconds


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Inference:   0%|          | 0/15 [00:00<?, ?it/s]

...

Inference:   0%|          | 0/15 [00:00<?, ?it/s]

prediction time : 0.7952795028686523 seconds


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Inference:   0%|          | 0/15 [00:00<?, ?it/s]

prediction time : 0.8261926174163818 seconds
mean time : 0.0826568841934204 seconds


# Finetuning a Model on own data

In [4]:
list_of_dict_results = []

In [11]:
seed = 6822
    

args = TrainingArgs(
    output_dir='FCoref',
    overwrite_output_dir=True,
    model_name_or_path='biu-nlp/f-coref',
    device='cuda:0',
    epochs = 50,
    logging_steps=100,
    eval_steps=100,
    seed = seed
)   # you can control other arguments such as learning head and others.


trainer = CorefTrainer(
args = args,
train_file = f'/kaggle/input/tdac-coref-corpus/TDAC_corpus_train.jsonlines', 
dev_file='/kaggle/input/tdac-coref-corpus/TDAC_dev_test_combined.jsonlines',    # optional
test_file = f'/kaggle/input/tdac-coref-corpus/TDAC_dev_test_combined.jsonlines'   # optional
#nlp=nlp # optional, for custom nlp class from spacy
)

eval_loss,▁▃▇█
f1,▁██▃
loss,█▁▁▁
mention f1,▁█▆▆
mention precision,█▅▇▁
mention recall,▁█▆▇
post pruning mention f1,▁▁▁█
post pruning mention precision,▁▁▁█
post pruning mention recall,▁▁▁█
precision,█▅█▁
recall,▁█▇▆


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [12]:
train_start = time.time()
trainer.train()
print(f'training time: {time.time() - train_start} seconds')

Creating batches for training: 9it [00:00, 81.05it/s]


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9 [00:00<?, ?it/s]

training time: 102.83800649642944 seconds


In [13]:
start = time.time()
res = trainer.evaluate(test = True)
list_of_dict_results.append(res)

Inference:   0%|          | 0/15 [00:00<?, ?it/s]

In [14]:
list_of_dict_results

[{'eval_loss': 0.13345478847622871,
  'post pruning mention precision': 0.02332089552238806,
  'post pruning mention recall': 0.9803921568627451,
  'post pruning mention f1': 0.04555808656036446,
  'mention precision': 0.8235294117647058,
  'mention recall': 0.5490196078431373,
  'mention f1': 0.6588235294117647,
  'precision': 0.6799674182027123,
  'recall': 0.45353535353535346,
  'f1': 0.5440728015936155},
 {'eval_loss': 0.1383015662431717,
  'post pruning mention precision': 0.02378731343283582,
  'post pruning mention recall': 1.0,
  'post pruning mention f1': 0.04646924829157175,
  'mention precision': 0.8235294117647058,
  'mention recall': 0.5490196078431373,
  'mention f1': 0.6588235294117647,
  'precision': 0.6799674182027123,
  'recall': 0.45353535353535346,
  'f1': 0.5440728015936155}]

In [39]:
precision, recall, f1 = [], [], []

for d in list_of_dict_results:
    precision.append(d['precision'])
    recall.append(d['recall'])
    f1.append(d['f1'])
    



In [42]:
import numpy as np

0.6707890584361174

In [43]:
print(f'Mean Precision: {np.mean(precision)} (+/-) {np.std(precision)}')
print(f'Mean Recall: {np.mean(recall)} (+/-) {np.std(recall)}')
print(f'Mean F1: {np.mean(f1)} (+/-) {np.std(f1)}')

Mean Precision: 0.6707890584361174 (+/-) 0.004589179883297501
Mean Recall: 0.4385627705627705 (+/-) 0.007486291486291452
Mean F1: 0.5303325113383215 (+/-) 0.006870145127646987
